In [101]:
import pandas as pd

details = pd.read_excel("Data/Case_1.xlsx", sheet_name="Details")
history = pd.read_excel("Data/Case_1.xlsx", sheet_name="History Taking", keep_default_na=False)
physical = pd.read_excel("Data/Case_1.xlsx", sheet_name="Physical Examination")
diagnosis = pd.read_excel("Data/Case_1.xlsx", sheet_name="Differential Diagnosis")
Investigations = pd.read_excel("Data/Case_1.xlsx", sheet_name="Investigations")

In [133]:

from importlib import reload
import simulator
reload(simulator)

history_taking_example = "\n".join(
  [f"Question: {row['Question']} | Response: {row['Patient Response']}"
    for index, row in history.iterrows()])

case_generator = simulator.generators.CaseGenerator(
  model_name="gpt-4o-mini",
  case_stem = "Man comes to ED complaining of chest pain",
  basic_details_example= details["Case Prompt"].values[0],
  vitals_example = details["Vitals"].values[0],
  physical_presentation_example = details["Physical presentation"].values[0],
  challenging_question_example = details["Challenging Questions to Ask"].values[0],
  history_taking_example = history_taking_example
)

In [134]:
output = case_generator.generate_basic_details()

In [135]:
output

['Michael Green, a 52-year-old male, comes to the ED complaining of chest pain.',
 'BP: 150/95 mm Hg\nTemp: 99.1°F (37.3°C)\nRR: 20/minute\nHR: 88/minute, regular',
 None,
 "'Could this be something serious? What should I do next?'"]

In [136]:
output = case_generator.generate_history()

In [138]:
output

HistoryTakingChecklist(ChecklistItems=[HistoryTakingChecklistItem(question='Chief complaint', response='Chest pain'), HistoryTakingChecklistItem(question='Onset', response='Forty minutes ago'), HistoryTakingChecklistItem(question='Precipitating events', response='Nothing; I was sitting and suddenly felt this pain.'), HistoryTakingChecklistItem(question='Progression', response='Constant severity'), HistoryTakingChecklistItem(question='Severity on a scale of 1 to 10', response='My pain is 6 out of 10'), HistoryTakingChecklistItem(question='Location', response='Center of the chest, feels tight.'), HistoryTakingChecklistItem(question='Radiation', response='To my left arm and jaw.'), HistoryTakingChecklistItem(question='Quality', response='Tightness, like a band around my chest.'), HistoryTakingChecklistItem(question='Alleviating / Exacerbating factors', response='Nothing seems to help.'), HistoryTakingChecklistItem(question='Shortness of breath', response='Yes, I feel a bit short of breath

In [140]:
output = case_generator.generate_physical_exam()

In [144]:
output

PhysicalExamChecklist(ChecklistItems=[PhysicalExamChecklistItem(technique='Inspection of the chest', justification='To assess for any visible abnormalities, such as asymmetry, deformities, or signs of distress.'), PhysicalExamChecklistItem(technique='Palpation of the chest wall', justification='To identify any areas of tenderness, crepitus, or abnormal masses.'), PhysicalExamChecklistItem(technique='Auscultation of heart sounds', justification='To evaluate for any abnormal heart sounds, murmurs, or signs of ischemia.'), PhysicalExamChecklistItem(technique='Auscultation of lung sounds', justification='To check for any abnormal lung sounds, such as wheezing or crackles, which may indicate respiratory issues.'), PhysicalExamChecklistItem(technique='Assessment of peripheral pulses', justification='To evaluate circulation and check for any signs of peripheral vascular disease.'), PhysicalExamChecklistItem(technique='Blood pressure measurement in both arms', justification='To assess for any 

In [145]:
output = case_generator.generate_investigations()

In [146]:
output

InvestigationsChecklist(ChecklistItems=[InvestigationsChecklistItem(investigation='Electrocardiogram (ECG)', justification="To assess for any signs of ischemia or infarction given the patient's chest pain and risk factors."), InvestigationsChecklistItem(investigation='Cardiac biomarkers (e.g., troponin levels)', justification="To evaluate for myocardial injury or infarction, especially considering the patient's symptoms and family history."), InvestigationsChecklistItem(investigation='Chest X-ray', justification='To rule out other causes of chest pain such as pneumonia, pneumothorax, or aortic dissection.'), InvestigationsChecklistItem(investigation='Complete blood count (CBC)', justification="To check for any signs of infection or anemia that could contribute to the patient's symptoms."), InvestigationsChecklistItem(investigation='Basic metabolic panel (BMP)', justification='To assess electrolyte levels and kidney function, which can be important in the context of chest pain.'), Inves

In [147]:
output = case_generator.generate_ddx()

In [148]:
output

DdxChecklist(ChecklistItems=[DdxChecklistItem(diagnosis='Acute Coronary Syndrome (ACS)', justification="The patient's chest pain is central, tight, and radiates to the left arm and jaw, which are classic symptoms of ACS. The history of hypertension and family history of heart disease further support this diagnosis."), DdxChecklistItem(diagnosis='Pulmonary Embolism (PE)', justification='The patient presents with chest pain and shortness of breath, which could indicate a PE, especially considering the labored breathing and sweating.'), DdxChecklistItem(diagnosis='Aortic Dissection', justification="The sudden onset of severe chest pain and the nature of the pain could suggest an aortic dissection, particularly given the patient's age and history."), DdxChecklistItem(diagnosis='Gastroesophageal Reflux Disease (GERD)', justification='The patient has a history of similar episodes after heavy meals, which could indicate GERD, although the current episode is more severe.'), DdxChecklistItem(di

In [192]:
import pandas as pd

case_number = 1

details_df = pd.DataFrame()
details_df.at[0, 'Basic details'] = case_generator.basic_details
details_df.at[0, 'Vitals'] = case_generator.vitals
details_df.at[0, 'Physical presentation'] = case_generator.physical_presentation
details_df.at[0, 'Challenging question'] = case_generator.challenging_question
details_df.to_csv("Data/details/" + str(case_number) + ".csv")


In [193]:
columns = ["Question", "Patient Response"]
history_df = pd.DataFrame(columns=columns)

for item in case_generator.history.ChecklistItems:
  history_df = pd.concat(
    [
      history_df,
      pd.DataFrame(
        [[
          item.question, item.response
        ]], columns=columns)
    ]
  )

history_df.reset_index(drop=True)
history_df.to_csv("Data/history/" + str(case_number) + ".csv")

In [191]:
columns = ["Physical Exam", "Justification"]
physical_exam_df = pd.DataFrame(columns=columns)

for item in case_generator.physical_exam.ChecklistItems:
  physical_exam_df = pd.concat(
    [
      physical_exam_df,
      pd.DataFrame(
        [[
          item.technique, item.justification
        ]], columns=columns)
    ]
  )

physical_exam_df.reset_index(drop=True)
physical_exam_df.to_csv("Data/physical/" + str(case_number) + ".csv")

In [190]:
columns = ["Investigations", "Justification"]
investigations_df = pd.DataFrame(columns=columns)

for item in case_generator.investigations.ChecklistItems:
  investigations_df = pd.concat(
    [
      investigations_df,
      pd.DataFrame(
        [[item.investigation, item.justification
          ]], columns=columns)
    ]
  )

investigations_df.reset_index(drop=True)
investigations_df.to_csv("Data/investigations/" + str(case_number) + ".csv")

In [ ]:
columns = ["Diagnosis", "Justification"]
diagnoses_df = pd.DataFrame(columns=columns)

for item in case_generator.ddx.ChecklistItems:
  diagnoses_df = pd.concat([
    diagnoses_df,
    pd.DataFrame(
      [[
        item.diagnosis, item.justification
      ]], columns=columns)
  ])

diagnoses_df.reset_index(drop=True)
diagnoses_df.to_csv("Data/diagnosis/" + str(case_number) + ".csv")